In [ ]:
%run -i ../python/common.py

# The von Neumann Architecture

As we saw in the [introduction](./intro.ipynb) assembly code directly maps to the native machine code of a computer.  As such the assembly instructions directly allow a programmer to use and control the basic functions of the computer in order to get the computer to do what they want.   Whether that is to search or sort an array of numbers, play a music file or some other task.   

But to understand assembly programming we have to learn what the basic parts are of any computer are and how they work so we can understand what the assembly instructions allow us to do.    This is why understanding how computers and software work are one in the same as learning assembly programming. 

In [ ]:
display(Markdown(htmlFig("../images/edvac.png", 
              align="center", 
              margin="auto 0 0 auto", 
              width="100%", id="fig:edvac", 
             caption="Figure: The archetype of the general-purpose digital computer")))

In [ ]:
display(Markdown(htmlFig("../images/SLS_TheMachine.png", 
              width="80%", id="fig:vnm", 
             caption="<center>Figure: Our illustration of a von Neumann computer.  Our view is a slight up date that put interms of todays computers.</center>")))

Despite the fact that there are many different manufactures of computers they all largely 
share a basic common structure.  We call the generic components, their organization 
and the way they interact the architecture of a machine.  In our 
case the common architecture to which most programmable computers are built around 
is that of the von Neumann Architecture named after [
John von Neumann](https://en.wikipedia.org/wiki/John_von_Neumann).

> <img style="margin: 5px 0px -5px 0px;" align="left" width="60" src="../images/commentary.svg"> <p style="background-color:powderblue;"> In our journey to understand computers we will be exploring a fascinating story of human innovation and ingenuity.  This story is full of characters, such as John von Neumann, Alan Turning, Admiral Grace Hopper, Ada Love Lace, and many more, that dared to be and think differently.  Not only do we owe a great deal of gratitude to these courageous people, who often risked a lot in suggesting new ways of thinking and doing things, challenging the orthodoxy of their day, we can also draw inspiration in their diversity and bravery. Remember who you are and that your voice matters, tomorrows innovation rests on your actions!  

## The Central Processing Unit (CPU)

There are many words today that we might hear used to refer to the Central Processing Unit (CPU) including processor, micro-processor and core.  Our goal, at this point, is to build our knowledge of how all computers work and then dive into more details by looking at how a particular computer works.  From this more generic perspective there are two ways of considering what a CPU is.  

1. Physically: A complex electical device composed of [transistors](https://en.wikipedia.org/wiki/Transistor) and wires.
2. Logically: A core building block for programmable information processing 


### Physically

While we won't dwell too much on the physical nature of a CPU it is worth "looking" at a few examples and noting a few if their characteristics and various challenges we face in building more advanced versions.  These challenges arise from physical limitation in the construction, connection to other devices and powering of the CPU itself. These challenges and the current approachs to trying to mitigate them also has an effect on how one needs to write software that will perform well on modern hardware.  We don't really need to worry about that right now but it is a theme we will revist later once we have a core understanding of the basics of how software and hardware interact.

In [ ]:
display(Markdown(htmlFig("../images/physcpus.png", 
              align="left", 
              margin="auto 1em 0 auto", 
              width="100%", id="fig:physcpus", 
             caption="Figure: Examples of physical CPUs.  For each CPU we note the product name, the number of 'pins' that connect it to the reset of the computer and the count of transistors it is composed of.")))

In the <a href="#fig:physcpus">Examples of physical CPUs figure</a>, above, we see photos of some physical cpus along with a "pin out" diagram that describes how to physically connect it to the rest of the computer.  Over the years it is clear that the complexity of CPUs has certainly grown.  In the 1970's CPUs were composed of thousands of transistors and only required tens of pins to connect to rest of the computer.  By 2021 we see that cpus now contain billions of transistors and require thousands for physical connections to the reset of the system.

While an over-simplification, one of the uses of transistors in a CPU is to create, "switches" that from the logic circuts used to implement basic operations such as adding numbers. In order to make the CPU do these operations faster we need to be able to operate the transitors, turn them on and off, faster and faster.  Unfortunately doing so requires more and more energy and creates more more heat.  As such over the last decade it has become harder and harder to speed up CPUs, given the increase in energy and attendant increase in heat it leads too.   On the other hand we have managed to figure out how to continually shrink the size of transistors that has allowed us to pack more and more transistors into CPUs, however some believe that we are getting to end of this ability as we are approching the physical limits to how small we can make a transistor. 

Regardless we have reached a point that making cpus operate faster is very hard to do and while we might be able to fit more transitors on the chip we are at the limits of how many we have powered on at the same time (due to energy and heat constraints).  Finally it is also very hard to imagine that we can fit many more "pins" that connect the cpu to the rest of the computer.

Over the years these effects have resulted in a situation that the performance of the software we write has become more and more dependent on how it interacts with the internal way the extra transistors are used.  For example in a modern processor a large number of transistors are used to form what is called [cache](https://en.wikipedia.org/wiki/CPU_cache) memory.  While software can benefit, its performance increases, from caches it is possible to write the same software in different ways that get greater or lesser benefit if you understand how the caches work and the way your software interacts with them.  Similarly today CPU's often use the extra transistors to create multiple internal sub-CPUs, called [cores](https://en.wikipedia.org/wiki/Multi-core_processor).  A program, however does not automatically benefit from multiple cores unless it is written to explicity exploit them via "parallel threads" of execution.  But to understand how to do this one first needs to understand the classic model by which a single core CPU works and its interaction with memory.   

The bottom line is CPU's are complex organizations of transistors and as we reach the physical limits to how we traditionaly build them the way software is written and structured becomes more and more important with respect to our ability to get computers to do more.  But the first step to understanding how to do this is understanding the basics of the von Neumann model of software execution that CPUs implement.  

### Logically

So while physical CPU's have clearly gotten more and more complex their basic role and function as defined by the von Neumann architecture has remained more or less the same.   It is this precise property that makes it possible to learn the basics of how a generic CPU works and understand how software executes on any computer.  As a matter of fact if we consider the line of CPU's from INTEL software written on a 1970 verions of their products can still be run on the CPU's they produced in 2021 because at the heart of it the 2021 version still remains consistent with basic model of a computer built around the 1970's CPU. 

While the physical aspects of a CPU matter there is a logical internal structure and operation that define its programming model and this is what we need to care about so that we can get a precise idea of what it exactly means to execute software and therefore what software is exactly. The CPU has a set of internal components, functions a core "Loop" behaviour that combines its capabilities with the other two components of a computer; Memory and Input/Output (I/O) to execute programs. 

In what remains of this chapter we will build up our understanding of the von Neumann architecture outwards from the internals of the CPU and "The Loop" it implements outwards to the memory and I/O components.

## Visualizing a generic von Neumann computer

In order to understand a generic model of a von Neumann computer we will use a series of diagrams that progressively visualize the components and show how they interact to execute a program.      To be clear we are using a logical representation that refects a working model that captures the things we need to care about to program a computer and what it execution means.  Each part we introduce while generic maps to things that we will find refected in the "real" computers we will learn to practically program in this section of the book.  

At the end of this chapter we will look at how the SOL6502 simulated computer lets us look at the parts, manipulate them and watch them in action.  

### The CPU

In [ ]:
display(Markdown(htmlFig("../images/VNA_0.png",
                     align="left", 
                      margin="auto 1em 0 auto",
                     width="60%", id="fig:vna_0",
                     caption="Figure: The CPU") + '''
The cpu is a distinct physical region of the computer. Within it is housed several important parts that we need to understand.
'''                     ))

### Operations

In [ ]:
display(Markdown(htmlFig("../images/VNA_1.png",
                     align="right", 
                      margin="auto auto auto auto",
                     width="60%", id="fig:vna_1",
                     caption="Figure: CPU's contain a set of <em>M</em> operations that we will use in our programs. We label each operation with a unique number so that we can identify one from another.") + '''
Within the cpu there are circuits/components that provide a set of operations.  The exact set of operations and the number can differ widely between particular CPU brands and models (eg INTEL vs ARM).  However, commonly we can expect operations that 1) performance arithmetic and logic (eg. adding and comparing numbers), 2) move values in and out of the CPU, 3) Control what happens next. Collectivelly the ones that perform arithmetic and logic  are often referred to as the Arithmetic Logic Unit (ALU).  From our prespective the most important thing, is that the set of operations form the 'built' functions that we can use in our programs for the particular cpu we are working with. Given that the operations are CPU specific our programs will inherintly be CPU specific.   
'''                     ))

### Registers

> Registers are memory locations within the CPU that are connect to the operation circuits.  Each register has a unique fixed name and a value that can be changed.  Their values can be feed as input to an operation or set as as the output from an operation.   Typically three types categories of registers: 1) General Purpose Registers (GPRS) 2) Special Purpose Registers (SPRS) and 3) Hidden Registers.  

With respect to "normal" programming we largely only need to concern ourselves with the the GPRS and knowing about a couple of standard SPRS. As we will see soon it will also be useful to explicitly introduce at least one of the register that usually hidden from programmers to make understanding execution easier.  

> <img style="display: inline; margin: 1em 1em auto auto;" align="left" width="40px" src="../images/fyi.png"> <p style="background-color:powderblue;"> 
  What do we mean by "normal" programming?  As we saw in Part I: The UNIX Development Environment software is largley broken down into two parts: 1) application software that runs within processes and 2) a single operating system kernel composed of software that provides all processes with a wide range of special functions.  This includes the abiliy to create processes from an executable file and to share the computer among the processes.  Most CPU's provide a special mode of execution for the OS kernel.  In this mode an expanded set of operations and SPRS are available for use.  OS kernel software uses the GPRS, SPRS and the expanded set of operations to implement is routines.  We typically call this mode of operation "Priveliged" and as such we often call the kernel software priveledged code.  expanded access to the hardware resources of the computer and the application or user software as unprivedged.   Many CPUs expand on this idea to provide multiple levels of priviledge which enables a further degree of layering of the systems softwae.  For example may CPUs use extra levels of priviledge to introduction support for a virtual machine monitor (or hypervisor) layer of software that can sit below standard operating system kernels and allow the hardware to be shared by multiple OS's each thinking they are running on the computer by themselves.
    
We will visualize registers as boxes within the CPU.  Each box will be broken down so that the name of the register will be on left and the space for the value it contains on the right.

#### General Purpose Registers (GPRS)

In [ ]:
display(Markdown(htmlFig("../images/VNA_2.png",
                     align="left", 
                      margin="auto auto auto auto",
                     width="60%", id="fig:vna_2",
                     caption="Figure: CPU's contain a set of <em>M</em> operations that we will use in our programs.") + '''
                     '''                     ))

#### Instruction Register (IR)

##### OPCODE
> An OPCODE is an encoded value that identifies a particular operation and where it inputs come from and if output that is created should go too.  Inputs to an operation are typically called **Operands**.

#### Program Counter (PC)

In [ ]:
## Memmory

In [ ]:
display(Markdown(htmlFig("../images/edvacextract.png", 
              width="100%", id="fig:vnm", 
             caption="<center>Figure: Extract form EDVAC Report discussing Memory and its role.</center>")))

## The Loop -- Program Execution